<a href="https://colab.research.google.com/github/tutalae/BTC/blob/master/Lending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ftx

In [2]:
pip install ciso8601

In [3]:
pip install ccxt

In [4]:
import ftx, ccxt, hmac, datetime, pytz
from time import time, sleep
import urllib.parse
from typing import Optional, Dict, Any, List

from requests import Request, Session, Response
from ciso8601 import parse_datetime

# **LOGIN HERE !!!**

In [5]:
Public_URL = 'https://ftx.com/api'
ApiKey = "gtiu5hQbvzwhOYpCYH35xcjUQ8GySUUGNCw4jnN"
Secret = "X0C9xO2BJFARMUsxeWz6ABKz1xGk3sctMq1qGsg"
my_subaccount_nickname  = "BOTRebalanceXRP"

# log in 
client = ftx.FtxClient(Public_URL, ApiKey, Secret, my_subaccount_nickname)

In [6]:
class info:
  def __init__(self, symbol, total, value_usd, available):
    self.symbol = symbol
    self.total = total
    self.value_usd = value_usd
    self.available = available

  def show_balance(self):
     print("symbol : ", self.symbol,", total : ", self.total,", value_usd : ", self.value_usd,", available : ", self.available)

  def show_available(self):
    print(self.available)

class price:
  def __init__(self, pair , price, bid, ask, sizeIncrement):
    self.pair = pair
    self.price = price
    self.bid = bid
    self.ask = ask
    self.sizeIncrement = sizeIncrement

  def show_price(self):
      print("pair : ", self.pair,", price : ", self.price,", bid : ", self.bid,", ask : ", self.ask, ", sizeIncrement : ", self.sizeIncrement)

# **ATTENTION**
# 1) don't forget to check your coin in account
# 2) set : coin
# 3) set : lending_percentage

In [7]:
balance = client.get_balances()

print("you have {} type of coin in this account (*{}*)".format( len(balance),my_subaccount_nickname))

for i in range(len(balance)):
  print("index {} is {}".format(i, balance[i]["coin"]))

you have 5 type of coin in this account (*BOTRebalanceXRP*)
index 0 is EOSBULL
index 1 is USD
index 2 is XRPBEAR
index 3 is XRPBULL
index 4 is XRP


# **SET HERE !**

In [8]:
coin = 1 # 
rent_percentage = .6 # lending_percentage X/100 , 60% = 60/100 = .6

In [9]:
# if you have 7 coin then total port value = balance[0]["usdValue"] + balance[1]["usdValue"] + balance[2]["usdValue" ] + ... + balance[6]["usdValue"]
# until you have all 

In [10]:
def check():
  balance = client.get_balances()
  input_pair = "XRP/USD"
  
  pair = client.get_market(input_pair)
  balance = client.get_balances()
  input_pair_name = input_pair.replace('/', '')
  
  list_pair= price(str(input_pair), pair["price"], pair["bid"], pair["ask"], pair['sizeIncrement'])

  total_port_value  = balance[0]["usdValue"]+balance[1]["usdValue"]+balance[2]["usdValue"]+balance[3]["usdValue"]+balance[4]["usdValue"]
    
  print("total port value is {} USD".format(total_port_value))
  print("total {} is {}".format( balance[coin]["coin"] ,balance[coin]['total']))
  print("total {} available to borrow is {}".format(balance[coin]["coin"], balance[coin]['availableWithoutBorrow']))
  print("total {}% of {} amount is {}".format(balance[coin]["coin"] ,rent_percentage,balance[coin]['availableWithoutBorrow']*rent_percentage))

In [11]:
def lending(lend_amount):
  balance = client.get_balances()
  USDava = balance[coin]
  
  client.get_position("spot_margin/offer", {
    "coin" : balance[coin]["coin"],
    "size" : lend_amount , 
    "rate" : 1e-06
    })
  
  print("lending total {} usd from total".format(lend_amount))
  print(client._get("spot_margin/lending_info")[0])

In [12]:
def lending_condition(rent_percentage):

  if balance[coin]["availableWithoutBorrow"] > balance[coin]['total']*rent_percentage :
 # balance > rent_percentage (con 1)
    if balance[coin]["availableWithoutBorrow"] - balance[coin]['total']*(1-(rent_percentage)) > 1 :
      lending(balance[coin]['total']*rent_percentage)
      print("lending {} total {}".format(balance[coin]["coin"], balance[1]['total']*rent_percentage))

    else :
      print("{} available > {}% , No enough money".format(balance[coin]["coin"], rent_percentage*100))
    
  # balance < rent_percentage (con 2)
  elif balance[coin]["availableWithoutBorrow"] < balance[coin]['total']*rent_percentage :
    if balance[coin]["availableWithoutBorrow"] - balance[coin]['total']*(1-(rent_percentage)) > 1 :
      lending(balance[coin]["availableWithoutBorrow"] - balance[coin]['total']*(1-(rent_percentage)))
      print("lending {} total {}".format(balance[coin]["coin"], balance[1]["availableWithoutBorrow"] - balance[coin]['total']*(1-(rent_percentage))))

    else :
      print("{} available > {}% , No enough money".format(balance[coin]["coin"], balance[coin]['total']*(1-(rent_percentage))*100))
      pass

  # con 3
  else: 
    print("do nothing cause rate no change")

In [14]:
def main():
    try:
        print("-----------------start loop----------------------")
        check()

        my_date = datetime.datetime.now(pytz.timezone('Asia/Bangkok'))
        print("check price" , my_date, "@Bangkok time zome")
        print("take action")
        lending_condition(rent_percentage)
        
        check()
        print("-----------------end loop----------------------")
        print("-                                                       -")

    except :
        print('Error')
        pass

while True:
  sleep(30)
  main()
else:
  print("something error")

KeyboardInterrupt: ignored